In [1]:
#Packages to import
import pandas as pd
from scipy.stats import zscore
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

In [3]:
#Import collision data
df_collision=pd.read_csv("/Users/caoxinwei/Desktop/ML Collision Project/Collision Data.csv")

/var/folders/3b/rfp_35615lv8wlrkdpqtwrqw0000gn/T/ipykernel_13709/3307604499.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_collision=pd.read_csv("/Users/caoxinwei/Desktop/ML Collision Project/Collision Data.csv")


In [4]:
df_collision.head(2)

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Driver At Fault,Injury Severity,Circumstance,Driver Distracted By,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Latitude,Longitude,Location
0,DM8479000T,210020119,Takoma Park Police Depart,Property Damage Crash,05/27/2021 07:40:00 PM,NaN,NaN,NaN,IN PARKING LOT,NaN,NaN,OTHER,CLEAR,NaN,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,CF61C6EB-61C4-4627-9767-635D2F0174A6,No,NO APPARENT INJURY,NaN,UNKNOWN,MD,D3F480EE-638B-459A-BC20-533741BC4F76,SUPERFICIAL,ONE OCLOCK,OTHER,PARKED,NaN,0,No,Yes,2017,HINO,TWK,38.987657,-76.987545,"(38.98765667, -76.987545)"
1,MCP2970000R,15045937,MONTGOMERY,Property Damage Crash,09/11/2015 01:29:00 PM,NaN,NaN,NaN,Parking Lot: \n2525 Ennalls Ave,NaN,NaN,OTHER,CLEAR,NaN,DAYLIGHT,NO CONTROLS,UNKNOWN,NaN,E0C26FD8-EE66-4B8D-92AD-4AF5EBC8BF2C,Yes,NO APPARENT INJURY,NaN,UNKNOWN,NaN,14ECEFCD-3A58-4991-A9C6-A9D5CA54B419,UNKNOWN,UNKNOWN,PASSENGER CAR,PARKING,South,5,No,No,2012,TOYOTA,SU,39.039917,-77.053649,"(39.03991652, -77.05364898)"


In [5]:
###Exclude features we won't use
columns_to_keep = ['ACRS Report Type', 'Crash Date/Time', 'Route Type', 'Road Name', 'Cross-Street Name', 'Weather', 'Surface Condition', 'Light', 'Driver Substance Abuse', 'Speed Limit', 'Vehicle Year', 'Latitude', 'Longitude', 'Traffic Control','Collision Type', 'Driver At Fault', 'Vehicle Body Type', 'Vehicle Make', 'Vehicle Movement']
df_collision = df_collision[columns_to_keep]

In [6]:
df_collision.head(1)

,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Name,Weather,Surface Condition,Light,Driver Substance Abuse,Speed Limit,Vehicle Year,Latitude,Longitude,Traffic Control,Collision Type,Driver At Fault,Vehicle Body Type,Vehicle Make,Vehicle Movement
0,Property Damage Crash,05/27/2021 07:40:00 PM,NaN,NaN,NaN,CLEAR,NaN,DAYLIGHT,NONE DETECTED,0,2017,38.987657,-76.987545,NO CONTROLS,OTHER,No,OTHER,HINO,PARKED


In [7]:
df_collision = df_collision.dropna()

In [8]:
df_collision.shape

(107326, 19)

In [9]:
###1
# Drop rows where 'Speed Limit' equals 0
df_collision = df_collision[df_collision['Speed Limit'] != 0]

# Verify the rows are removed
print(f"Updated number of rows: {len(df_collision)}")

Updated number of rows: 106938


In [10]:
###2
# Perform Z-score normalization and overwrite the 'Speed Limit' column
df_collision['Speed Limit'] = zscore(df_collision['Speed Limit'])

# Verify the changes
print(df_collision['Speed Limit'].head())

5    -0.672483
6    -0.672483
8    -0.015398
9    -0.015398
10   -0.015398
Name: Speed Limit, dtype: float64


In [11]:
###3
# Ensure the column is parsed as datetime
df_collision['Crash Date/Time'] = pd.to_datetime(df_collision['Crash Date/Time'])

# Extract Day of the Week (0 = Sunday, 6 = Saturday)
df_collision['Day of Week'] = df_collision['Crash Date/Time'].dt.dayofweek

# Create a "Weekend" column (1 for weekend, 0 for weekdays)
df_collision['Weekend'] = df_collision['Day of Week'].apply(lambda x: 1 if x >= 5 else 0)

# Extract Hour of the Day (0-23)
df_collision['Time'] = df_collision['Crash Date/Time'].dt.hour

# Extract Month (0 = January, 11 = December)
df_collision['Month'] = df_collision['Crash Date/Time'].dt.month - 1  # Subtract 1 to make it 0-indexed

# Extract Year
df_collision['Year'] = df_collision['Crash Date/Time'].dt.year

# Extract Date in Month (1-31, no need to adjust as it's already correct)
df_collision['Date in Month'] = df_collision['Crash Date/Time'].dt.day

# Drop the "Crash Date/Time" column
df_collision = df_collision.drop(columns=['Crash Date/Time'])

# Display the updated DataFrame
df_collision.head()


,ACRS Report Type,Route Type,Road Name,Cross-Street Name,Weather,Surface Condition,Light,Driver Substance Abuse,Speed Limit,Vehicle Year,Latitude,Longitude,Traffic Control,Collision Type,Driver At Fault,Vehicle Body Type,Vehicle Make,Vehicle Movement,Day of Week,Weekend,Time,Month,Year,Date in Month
5,Injury Crash,Maryland (State),NORBECK RD,DRURY RD,CLOUDY,DRY,DAYLIGHT,NONE DETECTED,-0.672483,2010,39.116462,-77.050530,NO CONTROLS,SAME DIR REAR END,No,(SPORT) UTILITY VEHICLE,MERCEDES,SLOWING OR STOPPING,0,0,11,7,2023,28
6,Property Damage Crash,County,GREENTREE RD,OLD GEORGETOWN RD,CLEAR,DRY,DAYLIGHT,NONE DETECTED,-0.672483,2022,39.000144,-77.109881,NO CONTROLS,STRAIGHT MOVEMENT ANGLE,No,PASSENGER CAR,HOND,MOVING CONSTANT SPEED,3,0,12,6,2023,27
8,Property Damage Crash,Maryland (State),GEORGIA AVE,MAY ST,CLEAR,DRY,DARK LIGHTS ON,NONE DETECTED,-0.015398,2016,39.072460,-77.064860,TRAFFIC SIGNAL,SAME DIR REAR END,No,PASSENGER CAR,TOYOTA,STOPPED IN TRAFFIC LANE,4,0,20,10,2023,10
9,Property Damage Crash,Maryland (State),GEORGIA AVE,LINDELL ST,CLEAR,DRY,DARK LIGHTS ON,NONE DETECTED,-0.015398,2021,39.054407,-77.050488,NO CONTROLS,HEAD ON LEFT TURN,Yes,PASSENGER CAR,HONDA,MAKING LEFT TURN,0,0,19,9,2023,16
10,Property Damage Crash,Municipality,PERRY PKWY,ENT TO SHOPPING CENTER,CLOUDY,DRY,DAYLIGHT,NONE DETECTED,-0.015398,2002,39.148779,-77.213439,NO CONTROLS,STRAIGHT MOVEMENT ANGLE,No,PASSENGER CAR,HOND,MOVING CONSTANT SPEED,5,1,10,8,2023,30


In [13]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(
    df_collision,
    test_size=0.3,         # 30% for testing, 70% for training
    random_state=17,       
    stratify=df_collision['ACRS Report Type']             # Preserves class distribution in train and test
)

# Save the splits to CSV files
train_data.to_csv('collision_train_data.csv', index=False)  # Save training set
test_data.to_csv('collision_test_data.csv', index=False)    # Save testing set